In [1]:
import numpy as np
import qiskit
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.opflow import CircuitOp, CircuitStateFn
from qiskit import *
from qiskit.opflow.state_fns import StateFn
from qiskit.providers.aer import *
from qiskit.opflow.expectations import MatrixExpectation
from qiskit.opflow.converters import CircuitSampler
from qiskit_experiments.library import StateTomography
from qiskit_experiments.framework import ParallelExperiment
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import pennylane as qml
import torch
import torch.nn as nn
np.random.seed(31)

In [2]:

def fidel_calculator(data, n_qubits):
    fidelities = []
    fidelity = 0.0
    pj = []
    P_j = 0.0
    fidel_normal = []
    for j in range(n_qubits-1): #loop over batches
        for i in range(2**(2*n_qubits)):
            fidelity += np.sqrt(data[i][1][j]*data[i][2][j]) / np.sqrt(2**(2*n_qubits))
        fidelities.append(fidelity)
    for j in range(n_qubits-1):
        for i in range(2**(2*n_qubits)):
            P_j += data[i][1][j] **2 / 2**n_qubits
        pj.append(P_j)
    for j in range(n_qubits-1):
        a = fidelities[j] / pj[j] #since P_j = sum(a_j **2 / 2**n) = 1, we normalize fidelity using it
        fidel_normal.append(a)
    return fidel_normal

def data_generator(n_qubits):
    '''generates data for fidelity trainings
    returns data [list] containing elements, datum with structure:
    datum = [list of selected pauli operators,
            theoretical expectation values, 
            experimental expectation values,
            actual fidelity values ]
        '''
    data_size = 2**(2*n_qubits)
    data = []
    fidelity_actual = []
    operator_list = []
    fidelity = []
    expvals_theo = []
    expvals_exp = []
    diagnosis = []
    clbit = []
    for j in range(data_size):
        #theoretical expectation values
        paulis = ['I', 'X', 'Y', 'Z']
        W_j = [] # list of selected pauli ops
        wj = [] # list of selected paulis for training
        op = [] # turn into qiskit ops
        randomm = np.random.randint(4,size=n_qubits) # generate random number
        for i in range(n_qubits):
            ppp = str()
            for k in range(n_qubits):
                ppp += paulis[randomm[k]]
            wj.append(randomm[i])
            diagnosis.append(ppp)
            W_j.append(ppp) # using that random number, construct W_j
                
        for i in range(n_qubits):
            op.append(Operator(Pauli(W_j[i]))) #turn W_j into Operator list
            clbit.append(i)
        expvals_th = []
        pauli_tensor = []
        psi = QuantumCircuit(n_qubits) # arbitrary quantum state for expectation values
        psi = CircuitStateFn(psi)
        for i in range(n_qubits):
            circuit = QuantumCircuit(n_qubits)
            #circuit.h(0)
            circuit.append(op[i],[(i) for i in range(n_qubits)])
            pauli_tensor.append(CircuitOp(circuit))
            expval = psi.adjoint().compose(pauli_tensor[i]).compose(psi).eval().real
            expvals_th.append(expval)
        expvals_theo.append(expvals_th)
        
        
        ## experimental expvals    #### will be changed with ibmq  #######
        ''' since actual fidelities are requied, QST will be added to measurement setup '''
        
        expvals_ex = []
        for i in range(n_qubits):
            measure = StateFn(pauli_tensor[i], is_measurement=True).compose(psi)
            expectation = MatrixExpectation().convert(measure)
            sim = AerSimulator()
            sampler = CircuitSampler(sim).convert(expectation)
            expvals_ex.append(sampler.eval().real)
        expvals_exp.append(expvals_ex)
                
        # actual fidelities
        backend = AerSimulator()
        #generate list of pauli gates size of n_qubits
        tomopaulis = []
        for i in range(n_qubits):
            zzz = np.random.randint(5,size=1)
            if zzz == 0:
                tomopaulis.append('I')
            elif zzz == 1:
                tomopaulis.append('X')
            elif zzz == 2:
                tomopaulis.append('Y')
            else:
                tomopaulis.append('Z')

        gates = [qiskit.circuit.library.PauliGate(i) for i in tomopaulis]
        subexps = [StateTomography(gate, qubits=[i]) for i, gate in enumerate(gates)]
        tomoexp = ParallelExperiment(subexps)
        tomodata = tomoexp.run(backend, seed_simulation=100).block_for_results()

        fidel_actual = []
        for i, expdata in enumerate(tomodata.child_data()):
            state_result_i = expdata.analysis_results("state")
            fidelity_result_i = expdata.analysis_results("state_fidelity")
            fidel_actual.append(fidelity_result_i.value)
            
        #datum=[wj, , , fidel_actual]
        #data.append(datum)
        fidelity_actual.append(fidel_actual)
        operator_list.append(wj)
        
        
    return expvals_theo, expvals_exp, fidelity_actual, operator_list, diagnosis







fidelities = []
fidelity = 0.0
pj = []
P_j = 0.0
fidel_normal = []
data, fidel_actual = data_generator(5, 2**10)
print(len(data[0]))
for j in range(5-1): #loop over batches
    for i in range(2**(2*5)-1):
        fidelity += np.sqrt(data[0][1][j]*data[0][2][j]) / np.sqrt(2**(2*5))
    fidelities.append(fidelity)
for j in range(5-1):
    for i in range(2**(2*5)-1):
        P_j += data[i][1][j] **2 / 2**5
    pj.append(P_j)
for j in range(5-1):
    a = fidelities[j] / pj[j] #since P_j = sum(a_j **2 / 2**n) = 1, we normalize fidelity using it
    fidel_normal.append(a)

In [7]:
%%time
n_qubits = 2
expval_th, expval_exp, fidel_actual, op_list, diagnosis = data_generator(n_qubits)
print('diag')
print(diagnosis)
print('oplist')
print(fidel_actual)
xdata = np.concatenate([np.array([op_list], dtype=object), np.array([expval_exp], dtype=object)])
domain = np.linspace(0.9,1.0,122)
intervals = []
Ydata = np.array([fidel_actual], dtype=object)

def classifier():
    lst = []
    for j in range(n_qubits):
        for k in range(2**(2*n_qubits)):
            for i in range(len(domain)):
                if fidel_actual[k][j] < domain[i]:
                    lst.append(str(domain[i]))
    return lst
results = []
for f in fidel_actual:
    index = np.argmin(np.abs(domain - f))
    d = domain[index]
    result = fidel_actual_reshaped[index]
    results.append(result)




ydata = results
print(ydata)

diag
['XY', 'XY', 'XI', 'XI', 'XY', 'XY', 'XI', 'XI', 'IX', 'IX', 'YY', 'YY', 'IY', 'IY', 'YY', 'YY', 'XZ', 'XZ', 'ZZ', 'ZZ', 'XY', 'XY', 'YI', 'YI', 'YI', 'YI', 'IY', 'IY', 'ZX', 'ZX', 'ZZ', 'ZZ']
oplist
[[0.9980243147217881, 0.9998693978072786], [0.9996303846161662, 0.999596131945591], [0.9999923707800885, 0.9991429481018974], [0.9994050172531763, 0.9997179512364855], [0.9997769894997898, 0.9996551270379711], [0.9991960881899813, 0.9996322877488868], [0.9998379541768132, 0.9998703507372033], [0.9986540974250391, 0.9999570901801528], [0.9975387401937004, 0.9990708565475692], [0.9998236635789504, 0.9998122319831393], [0.9992198169239579, 0.9999847419093948], [0.9992720320517969, 0.9997693704835461], [0.9997779419013219, 0.9997617518157436], [0.9999075192557039, 0.9985822174479492], [0.9999446960665778, 0.9989182394004351], [0.9990073280513444, 0.9998093742067408]]


ValueError: operands could not be broadcast together with shapes (122,) (2,) 

In [4]:
print(len(fidel_actual))
print(len(xdata))

print(len(ydata))

#y_data = np.concatenate(np.array([ydata]), np.zeros([2]))
#y_data = np.vstack((ydata, np.zeros([3])))
batch_size = 2**(2*n_qubits)
print(len(intervals))
xtrain, ytrain = [], []
for i in range(int(batch_size*0.8)):
    [xtrain.append(xdata[j][i]) for j in range(n_qubits)]
    ytrain.append(ydata[i])
    
from sklearn.multiclass import OutputCodeClassifier
from sklearn.svm import LinearSVC

clf = OutputCodeClassifier(LinearSVC(random_state=0),code_size=4, random_state=0, n_jobs=1)
clf.fit(xdata, ydata).predict(xdata)

'''xtrain, xtest, ytrain, ytest = train_test_split(xdata, ydata, test_size=0.2, random_state=31)

from sklearn.tree import DecisionTreeClassifier
dtree_model = DecisionTreeClassifier(max_depth=3).fit(xtrain, ytrain)
dtree_predictions = dtree_model.predict(xtest)

cm = confusion_matrix(ytest, dtree_predictions)'''

16
2
37
0


ValueError: Found array with dim 3. Estimator expected <= 2.

In [ ]:

domain = np.linspace(0.8,1.0,122)
labels = []
intervals = []
for i in range(len(domain)):
    labels.append(str(domain[i]))
    intervals.append(domain[i])

print(a[0][1][1])
#print(a[0][1]) # 0th sample, 1st column, 4th element
def labeler(data, n_qubits, qubit_location):
    _fidelity = 0.0
    P_j = 0.0
    for i in range(2**10-1):
        _fidelity += np.sqrt(a[i][1][qubit_location]*a[i][2][qubit_location]) / np.sqrt(2**10)
    
    for i in range(2**10-1):
        P_j += a[i][1][2] **2 / 2**5
    fidel_normalized = _fidelity / P_j
    for i in range(len(domain)-1):
        if domain[i] < fidel_normalized < domain[i+1]:
            return intervals[i]


print(labeler(a, 5, 0), f'+- {0.2/122}')


In [ ]:
# importing necessary libraries
from sklearn import datasets
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split

#print(X)
#print(type(a))
#b = np.asarray(a[1], dtype=object)
#print(b)

In [ ]:
## quantum neural network


n_qubits = 5
q_depth = 6
dev = qml.device('default.qubit', wires=n_qubits)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
q_delta = 0.01
print('is cuda available? ',torch.cuda.is_available())

# quantum layers
def H_layer(nqubits):
    for i in range(nqubits):
        qml.Hadamard(wires=i) #superposing states
def RY_layer(theta_):
    for i, th in enumerate(theta_):
        qml.RY(th, wires=i) #rotating states
def entangler_layer(nqubits): # layer of cnots cross-firing. cnot[0,1]-cnot[1,2]-cnot[2,3]-...
    for i in range(0,nqubits-1, 2):  # evens
        qml.CNOT(wires=[i, i+1])
    for i in range(1,nqubits-1, 2): # odds
        qml.CNOT(wires=[i, i+1])   #entangling states
    
# variational circuit
dev = qml.device('default.qubit', wires=n_qubits)
@qml.qnode(dev, interface='torch')

def q_net(theta, q_weights_flat):
    q_weights = q_weights_flat.reshape(q_depth,n_qubits)
    H_layer(n_qubits) #superpose
    RY_layer(theta)   #embed features in qnode
    for i in range(q_depth):  #sequence of trainable variatinal layers
        entangler_layer(n_qubits)
        RY_layer(q_weights[i])
    exp_vals = [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)] 
    print('exp.vals = ', tuple(exp_vals))
    
    return tuple(exp_vals)

class dressedQnetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.preprocess = nn.Linear(512, n_qubits)
        self.preprocess = nn.Dropout(p=0.5)
        self.thetas = nn.Parameter(q_delta * torch.randn(q_depth * n_qubits))
        self.postprocess = nn.Linear(n_qubits, 2)

    def forward(self, input_features):
        print('features: ', input_features)
        print(len(input_features[0]))
        output_pre = self.pre_net(input_features)
        theta = torch.tanh(output_pre) * np.pi / 2.0  #nonlinear activation on q.layer

        output_q = torch.Tensor(0, n_qubits)
        output_q = output_q.to(device)
        for th in theta:
            q_out = q_net(th, self.thetas).float().unsqueeze(0)
            output_q = torch.cat((output_q, q_out))

        return self.postprocess(output_q)